In [1]:
import torchvision
from PIL import Image

/Users/xni/vision/torchvision/io/_video_opt.py:17: UserWarning: video reader based on ffmpeg c++ ops not available
  warnings.warn("video reader based on ffmpeg c++ ops not available")


In [2]:
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import numpy as np

In [3]:
from torchvision.datasets import MNIST

In [4]:
mnist = MNIST(root="/Users/xni/Documents/pytorch_task/",download=True)

In [5]:
mnist[0][0]

In [42]:
img_cat = Image.open("/Users/xni/Documents/pytorch_task/cat.jpg")

In [43]:
img_cat.show()

In [8]:
transformations = transforms.Compose([
    transforms.CenterCrop(224),
])

In [44]:
def adjust_hue(img, hue_factor):
    """Adjust hue of an image.

    The image hue is adjusted by converting the image to HSV and
    cyclically shifting the intensities in the hue channel (H).
    The image is then converted back to original image mode.

    `hue_factor` is the amount of shift in H channel and must be in the
    interval `[-0.5, 0.5]`.

    See `Hue`_ for more details.

    .. _Hue: https://en.wikipedia.org/wiki/Hue

    Args:
        img (PIL Image): PIL Image to be adjusted.
        hue_factor (float):  How much to shift the hue channel. Should be in
            [-0.5, 0.5]. 0.5 and -0.5 give complete reversal of hue channel in
            HSV space in positive and negative direction respectively.
            0 means no shift. Therefore, both -0.5 and 0.5 will give an image
            with complementary colors while 0 gives the original image.

    Returns:
        PIL Image: Hue adjusted image.
    """
    if not(-0.5 <= hue_factor <= 0.5):
        raise ValueError('hue_factor is not in [-0.5, 0.5].'.format(hue_factor))

#     if not _is_pil_image(img):
#         raise TypeError('img should be PIL Image. Got {}'.format(type(img)))

    input_mode = img.mode
    if input_mode in {'L', '1', 'I', 'F'}:
        return img

    h, s, v = img.convert('HSV').split()
    print(h,s, v)
    np_h = np.array(h, dtype=np.uint8)
    # uint8 addition take cares of rotation across boundaries
    with np.errstate(over='ignore'):
        np_h += np.uint8(hue_factor * 255)
    h = Image.fromarray(np_h, 'L')
    img = Image.merge('HSV', (h, s, v)).convert(input_mode)
    return img

In [45]:
img_new = adjust_hue(img_cat, 0.2)

<PIL.Image.Image image mode=L size=838x559 at 0x7FCB3985E490> <PIL.Image.Image image mode=L size=838x559 at 0x7FCB3985EE10> <PIL.Image.Image image mode=L size=838x559 at 0x7FCB3985E9D0>


In [46]:
img_new.show()

In [47]:
transform = transforms.ToTensor()
img_cat = transform(img_cat)

In [48]:
img_cat.shape # CHW -> NCHW

torch.Size([3, 559, 838])

In [50]:
img_cat = img_cat.unsqueeze(0)
img_cat.shape

torch.Size([1, 3, 559, 838])

In [51]:
img_cat[0,:,0,0]

tensor([0.2157, 0.2118, 0.2667])

In [52]:
import math

def hsv2rgb(h, s, v):
    h = float(h)
    s = float(s)
    v = float(v)
    h60 = h / 60.0
    h60f = math.floor(h60)
    hi = int(h60f) % 6
    f = h60 - h60f
    p = v * (1 - s)
    q = v * (1 - f * s)
    t = v * (1 - (1 - f) * s)
    r, g, b = 0, 0, 0
    if hi == 0: r, g, b = v, t, p
    elif hi == 1: r, g, b = q, v, p
    elif hi == 2: r, g, b = p, v, t
    elif hi == 3: r, g, b = p, q, v
    elif hi == 4: r, g, b = t, p, v
    elif hi == 5: r, g, b = v, p, q
    r, g, b = int(r * 255), int(g * 255), int(b * 255)
    return r, g, b

def rgb2hsv(r, g, b):
    r, g, b = r/255.0, g/255.0, b/255.0
    mx = max(r, g, b)
    mn = min(r, g, b)
    df = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        h = (60 * ((g-b)/df) + 360) % 360
    elif mx == g:
        h = (60 * ((b-r)/df) + 120) % 360
    elif mx == b:
        h = (60 * ((r-g)/df) + 240) % 360
    if mx == 0:
        s = 0
    else:
        s = df/mx
    v = mx
    return h, s, v

In [53]:
img_cat.shape

torch.Size([1, 3, 559, 838])